# Spark DataFrame - Missing Data

In [1]:
from pyspark.sql import SparkSession

In [2]:
# chamei esta sessao de "miss" mas voce pode chamar do que quiser.
spark = SparkSession.builder.appName('miss').getOrCreate()

In [3]:
# criei um dataframe a partir do arquivo ContainsNull.csv
df = spark.read.csv('./recursos/Spark_DataFrames/ContainsNull.csv', 
                    header=True, inferSchema=True)

In [4]:
# vamos mostrar como se parece
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



As colunas Name e Sales possuem valores nulos.

### Dropping the missing data

Vamos falar sobre apagar dados faltantes.

Para apagar todas as linhas que possuem dados faltantes.

In [6]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



Mas voce tambem pode especificar um threshold para um numero minimo de campos nao nulos.

In [7]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Veja que a linha "emp2" que tinha apenas um campo nao nulo foi retirado.

Outro parametro "how" cujo valor default "any" retira todas as linhas que contem valores nulos.

In [8]:
df.na.drop(how="any").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



O valor "all" retira apenas as linhas que contem TODOS os valores nulos.

In [9]:
df.na.drop(how="all").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Outro parametro "subset" considera retirar apenas as linhas 
cujos valores nulos estejam na coluna definida por este parametro.

In [10]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Veja que considerou apenas as linhas cujo campo "Sales" era nulo.

### Filling the missing values

As vezes voce nao quer apagar as linhas que possuem valores nulos, mas quer preencher os valores nulos.

In [11]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [12]:
df.na.fill('FILL VALUE').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL VALUE| null|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



Veja que todos os campos 'string' com valores nulos foram preenchidos com a string 'FILL VALUE'.

In [13]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Agora todos os campos 'double' com valores nulos foram preenchidos com o valor 0.

Podemos usar o parametro "subset" da mesma forma para especificar qual coluna queremos preencher os valores nulos.

In [15]:
df.na.fill('No name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No name| null|
|emp3|No name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



Agora quero preencher os valores nulos da coluna "Sales" com a media dos valores nao nulos.

In [16]:
# importar a funcao "mean"
from pyspark.sql.functions import mean

In [18]:
# extrair o valor medio da coluna "Sales"
mean_val = df.select(mean(df['Sales'])).collect()
mean_val

[Row(avg(Sales)=400.5)]

In [21]:
# Queremos apenas o numero que representa o valor medio da coluna "Sales".
mean_val[0][0]

400.5

In [22]:
mean_sales = mean_val[0][0]
mean_sales

400.5

In [23]:
# preenchemos os valores nulos com a media obtida.
df.na.fill(mean_sales,['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Podemos fazer tudo isso em apenas uma linha.

In [25]:
df.na.fill(
    df.select(mean(df['Sales'])).collect()[0][0],
    ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

